## Third Movement: 

In [1]:
# Import required libraries
import networkx as nx
import numpy as np
from scipy.special import factorial
from datetime import datetime
import logging
from scipy.stats import geom
import sys
import os

In [2]:
import os
experiment_log_file = 'experiment_3.log'
log_path = os.path.join(os.getcwd(), experiment_log_file)

if os.path.exists(log_path):
    os.remove(log_path)

logger = logging.getLogger()
handler = logging.FileHandler(log_path, mode='w')
handler.setFormatter(logging.Formatter('%(message)s'))
logger.addHandler(handler)
logger.setLevel(logging.INFO)

logger.info(f'[1] {experiment_log_file}')
logger.info(f'[1] "{datetime.now().strftime("%a %b %d %H:%M:%S %Y")}"')

In [3]:
%run attack_graph_MIR100.ipynb

### Use Third Distribution

In [4]:
attack_rate_list = [0]   
defense_rate_list = [0]  

In [5]:
def random_steps(route, attack_rate=None, defense_rate=None, graph=None):
    #print("\n=== Debug: random_steps function ===")
    #print("Input route:", route)
    
    hardness = []
    #print("\nCollecting edge probabilities:")
    
    # Get probabilities for each edge in route
    for i in range(len(route) - 1):
        u, v = route[i], route[i+1]
        edges_dict = graph[u][v]
        
        #print(f"\nEdge {u} -> {v}:")
        #print("Raw edge data:", edges_dict)

        # Simplified edge weight extraction
        if isinstance(edges_dict, float):
            weight = edges_dict
        else:
            # Get all weights for parallel edges
            weights = []
            for _, attr in edges_dict.items():
                if isinstance(attr, dict):
                    weights.append(attr.get('weight', 0.0))
                elif isinstance(attr, float):
                    weights.append(attr)
            
            weight = max(weights) if weights else 0.0
        
        prob = np.exp(-weight)
        hardness.append(prob)
    
    #print("\nFinal hardness array:", hardness)
    
    # Convert to numpy array and handle NaN values
    hardness = np.nan_to_num(np.array(hardness), nan=1.0)
    #print("Hardness after nan handling:", hardness)
    
    # Calculate probabilities
    cumulative_probs = np.concatenate(([1.0], np.cumprod(hardness)))
    stop_probs = np.concatenate((1 - hardness, [1.0]))
    
    #print("Cumulative probabilities:", cumulative_probs)
    #print("Stop probabilities:", stop_probs)
    
    # Calculate final PDF
    pdf = cumulative_probs * stop_probs
    #print("PDF before normalization:", pdf)
    
    # Normalize PDF
    pdf_sum = pdf.sum()
    if pdf_sum < 1e-15:
        pdf[:] = 1e-7
        pdf_sum = pdf.sum()
    
    final_pdf = pdf / pdf_sum
    #print("Final normalized PDF:", final_pdf)
    #print("=== End random_steps debug ===\n")
    
    return final_pdf

In [6]:
# %run ctr-core_simple.ipynb
%run ctr-core_tests.ipynb
main()


After merging targets:
Nodes: [1, 5, 15, 11, 3, 6, 8, 4, 7, 2, 9, 10, 0, 'c(12,13,14,16)']
Edges with weights:
1 -> 5 (key=0) : 2.1958405355640576
5 -> 15 (key=0) : 2.1958405355640576
15 -> c(12,13,14,16) (key=0) : 0.7489220813074156
15 -> c(12,13,14,16) (key=1) : 0.7489220813074156
15 -> c(12,13,14,16) (key=2) : 0.7489220813074156
11 -> c(12,13,14,16) (key=0) : 1.064439873679208
11 -> c(12,13,14,16) (key=1) : 0.7489220813074156
11 -> c(12,13,14,16) (key=2) : 0.0
3 -> 6 (key=0) : 1.064439873679208
3 -> 8 (key=0) : 0.7489220813074156
6 -> 8 (key=0) : 0.0
8 -> c(12,13,14,16) (key=0) : 0.0
8 -> c(12,13,14,16) (key=1) : 0.7489220813074156
8 -> 10 (key=0) : 0.0
4 -> 7 (key=0) : 0.7489220813074156
7 -> c(12,13,14,16) (key=0) : 0.7489220813074156
7 -> c(12,13,14,16) (key=1) : 0.7489220813074156
7 -> 10 (key=0) : 0.7489220813074156
2 -> c(12,13,14,16) (key=0) : 1.064439873679208
2 -> 9 (key=0) : 0.7489220813074156
2 -> 10 (key=0) : 1.064439873679208
2 -> 11 (key=0) : 0.7489220813074156
9 -> c

In [7]:
with open(experiment_log_file, 'r') as f:
    print(f.read())

[1] experiment_3.log
[1] "Thu Jan 09 13:24:59 2025"

++++++++++++++++++++++++++++++++

The virtual target nodeID is c(12,13,14,16)

attack rate =  0 , defense rate =  0 

	equilibrium for multiobjective security game (MOSG)

optimal defense strategy:
         prob.
10 0.000000e+00
11 0.000000e+00
15 4.799774e-01
5 0.000000e+00
6 0.000000e+00
7 9.098605e-02
8 4.290366e-01
9 0.000000e+00

worst case attack strategies per goal:
          1
1 0.0000000
2 0.3278186
3 0.0000000
4 0.0000000
5 0.0000000
6 0.3871037
7 0.2850777
8 0.0000000
9 0.0000000
10 0.0000000
11 0.0000000
[1] 0.053

Defender can keep attacker success below: 0.053
Attacker can guarantee success probability of: 0.053

